## Importando Bibliotecas

In [27]:
import tensorflow as tf 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from keras.optimizers import Adam
from keras.models import Model
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## Pre-processamento dos dados
**Carregando os dados de treino e teste:**

In [2]:
train_images = pd.read_csv('../fashion-mnist_train.csv')
test_images = pd.read_csv('../fashion-mnist_test.csv')

**Separando as labels do conjuntos:**

In [3]:
train_labels=train_images.loc[:, ['label']]
train_images=train_images.drop(['label'], axis=1)
test_labels=test_images.loc[:, ['label']]
test_images=test_images.drop(['label'], axis=1)

**Normalizando os data-sets:**

In [4]:
train_images = train_images / 255.0
test_images = test_images / 255.0


----------------------
## Construção do modelo base
**Esta rede será utilizada posteriomente para comparações entre os modelos de dimensionalidade reduzida utilizando PCA e autoencoder.**

**Definição da estrutura do modelo por meio do Keras Sequential:**

In [5]:
model = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(784)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 59,850
Trainable params: 59,850
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Treino do modelo:**

In [7]:
model.fit(train_images, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 6s 124us/sample - loss: 0.5209 - accuracy: 0.8152 - val_loss: 0.4132 - val_accuracy: 0.8530
Epoch 2/5
48000/48000 [==============================] - 6s 121us/sample - loss: 0.3867 - accuracy: 0.8570 - val_loss: 0.3680 - val_accuracy: 0.8652
Epoch 3/5
48000/48000 [==============================] - 6s 116us/sample - loss: 0.3508 - accuracy: 0.8719 - val_loss: 0.3848 - val_accuracy: 0.8627
Epoch 4/5
48000/48000 [==============================] - 5s 98us/sample - loss: 0.3276 - accuracy: 0.8792 - val_loss: 0.3647 - val_accuracy: 0.8732
Epoch 5/5
48000/48000 [==============================] - 6s 126us/sample - loss: 0.3088 - accuracy: 0.8863 - val_loss: 0.3300 - val_accuracy: 0.8842


**Verificação dos resultados do modelo:**

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 40us/sample - loss: 0.3224 - accuracy: 0.8811
Model - 3 layers - test loss: 32.237203629016875
Model - 3 layers - test accuracy: 88.10999989509583


-----------------------------
## Redução de Dimensionalidade Usando Principal component analysis (PCA)

**Definição do número de componentes:**

In [9]:
pca = PCA(n_components=3)

**Construção dos novos datasets de treino e de teste com dimensões reduzidas usando o PCA:**

In [10]:
train_images_r = pca.fit(train_images).transform(train_images)
test_images_r = pca.fit(test_images).transform(test_images)

**Alteração do tipo dos dados de numpy array para pandas dataframe:**

In [11]:
train_images_r = pd.DataFrame(data=train_images_r)
test_images_r= pd.DataFrame(data=test_images_r)

**Definição da estrutura do modelo por meio do Keras Sequential que receberá os dados com dimensões reduzidas pelo PCA:**

In [12]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(3)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                256       
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 9,866
Trainable params: 9,866
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [13]:
model_r.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Treino do modelo:**

In [14]:
model_r.fit(train_images_r, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 59us/sample - loss: 0.9848 - accuracy: 0.5941 - val_loss: 0.9172 - val_accuracy: 0.6154
Epoch 2/5
48000/48000 [==============================] - 3s 54us/sample - loss: 0.8961 - accuracy: 0.6240 - val_loss: 0.8983 - val_accuracy: 0.6109
Epoch 3/5
48000/48000 [==============================] - 2s 50us/sample - loss: 0.8787 - accuracy: 0.6352 - val_loss: 0.8878 - val_accuracy: 0.6264
Epoch 4/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.8705 - accuracy: 0.6371 - val_loss: 0.8760 - val_accuracy: 0.6387
Epoch 5/5
48000/48000 [==============================] - 2s 51us/sample - loss: 0.8628 - accuracy: 0.6416 - val_loss: 0.8657 - val_accuracy: 0.6405


**Verificação dos resultados do modelo:**

In [15]:
test_loss, test_acc = model_r.evaluate(test_images_r, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 19us/sample - loss: 0.8679 - accuracy: 0.6325
Model - 3 layers - test loss: 86.78787710189819
Model - 3 layers - test accuracy: 63.24999928474426


------------------------------------------------------

## Redução de Dimensionalidade Usando Autoencoders

In [16]:
# this is the size of our encoded representations
encoding_dim = 2
input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
encoder = Model(input_img, encoded)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
encoded_input = Input(shape=(encoding_dim,))
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [17]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [18]:
#REMOVER troque o 'adadelta' pelo 'adam'
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [19]:
autoencoder.fit(train_images, train_images,
                epochs=5,
                shuffle=True,
                validation_data=(test_images, test_images))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 126us/step - loss: 0.4339 - val_loss: 0.4070
Epoch 2/5
60000/60000 [==============================] - 7s 122us/step - loss: 0.4003 - val_loss: 0.3934
Epoch 3/5
60000/60000 [==============================] - 7s 121us/step - loss: 0.3897 - val_loss: 0.3848
Epoch 4/5
60000/60000 [==============================] - 7s 125us/step - loss: 0.3823 - val_loss: 0.3787
Epoch 5/5
60000/60000 [==============================] - 7s 122us/step - loss: 0.3784 - val_loss: 0.3765


In [20]:
encoded_imgs_train = encoder.predict(train_images)
encoded_imgs_test = encoder.predict(test_images)


In [21]:
train_images_a = pd.DataFrame(data=encoded_imgs_train)
test_images_a =pd.DataFrame(data=encoded_imgs_test)

In [22]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(2)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 9,802
Trainable params: 9,802
Non-trainable params: 0
_________________________________________________________________


In [23]:
model_r.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
model_r.fit(train_images_a, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 56us/sample - loss: 1.2776 - accuracy: 0.4770 - val_loss: 1.1652 - val_accuracy: 0.5298
Epoch 2/5
48000/48000 [==============================] - 3s 59us/sample - loss: 1.1591 - accuracy: 0.5273 - val_loss: 1.1587 - val_accuracy: 0.5247
Epoch 3/5
48000/48000 [==============================] - 3s 54us/sample - loss: 1.1415 - accuracy: 0.5338 - val_loss: 1.1358 - val_accuracy: 0.5401
Epoch 4/5
48000/48000 [==============================] - 3s 57us/sample - loss: 1.1327 - accuracy: 0.5375 - val_loss: 1.1280 - val_accuracy: 0.5381
Epoch 5/5
48000/48000 [==============================] - 2s 51us/sample - loss: 1.1292 - accuracy: 0.5388 - val_loss: 1.1270 - val_accuracy: 0.5425


In [25]:
test_loss, test_acc = model_r.evaluate(test_images_a, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 18us/sample - loss: 1.1251 - accuracy: 0.5438
Model - 3 layers - test loss: 112.50503034591675
Model - 3 layers - test accuracy: 54.37999963760376


------------------------------------------------------

## Clustering com K-means

Optamos pela utilização das imagens reduzidas por meio do PCA, já que obtiveram os melhores resultados de predição na rede.

In [26]:
train_images_k = train_images_r
test_images_k = test_images_r

In [36]:
kmean_model = KMeans(n_clusters=10)
kmean_model.fit(train_images_k)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [37]:
kmean_model.cluster_centers_

array([[-6.27051092, -0.13563267,  1.32175529],
       [ 5.22021128, -2.52884828, -1.09440373],
       [ 7.09581384,  2.84302316,  1.54863414],
       [ 1.04541615, -4.99358442, -1.63890707],
       [-1.46687273,  4.85195766, -4.08896511],
       [-0.58298877, -0.3012933 ,  1.95696813],
       [ 3.62148213,  1.29958684,  2.02530586],
       [-2.42214801, -3.87270357,  0.43116225],
       [ 1.26143436,  5.40593945, -0.41681487],
       [-4.61510826,  2.87519023, -0.75452347]])

---------------------------------
## Clustering com o DBScan
